In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
#----jdfg
def generate_pro_session_dataset(target_rows=1550127):
    print(f"🚀 Generando versión final con Payment Method garantizado en Purchase...")
    
    event_master = {
        'home_page': 1, 'log_in': 2, 'page_view': 3, 'scroll': 4, 
        'click_product': 5, 'add_to_cart': 6, 'begin_checkout': 7, 
        'end_checkout': 8, 'purchase':  9  
    }
    
    funnel_steps = list(event_master.keys())
    tz_map = {'Chile': 'UTC-3', 'México': 'UTC-6', 'España': 'UTC+1', 'Argentina': 'UTC-3', 'Colombia': 'UTC-5'}
    pay_methods = ['VISA', 'MASTERCARD', 'CREDIT_CARD', 'DEBIT_CARD', 'BANK_TRANSFERENCE', 'ONTOP', 'CRIPTOMONEDA', 'DIGITAL_WALLET']
    pay_weights = [0.25, 0.25, 0.15, 0.15, 0.05, 0.05, 0.05, 0.05]
    
    all_data = []
    base_date = datetime(2026, 1, 1)
    n_users = 300000 
    row_id = 1 

    for i in range(n_users):
        u_id = 11000000 + i
        u_country = np.random.choice(list(tz_map.keys()))
        u_tz = tz_map[u_country]
        u_device = np.random.choice(['Mobile', 'Desktop', 'Tablet'])
        u_source = np.random.choice(['Google', 'Facebook', 'Direct', 'Email'])
        
        for s in range(np.random.randint(1, 5)):
            u_session_id = f"SESS-{u_id}-{s+1}"
            u_category = np.random.choice(['Electronics', 'Clothing', 'Home', 'Beauty'])
            u_sku = f"SKU-{np.random.randint(1000, 5000)}"
            u_price = round(np.random.uniform(20.0, 500.0), 2)
            u_cart_id = f"CART-{np.random.randint(100000, 999999)}"
            
            # Decidimos el pago para la sesión por si llega a comprar
            session_payment = np.random.choice(pay_methods, p=pay_weights)
            
            u_time = base_date + timedelta(days=np.random.randint(0, 31), hours=np.random.randint(0, 24))
            n_steps = np.random.choice(range(1, 10), p=[0.2, 0.15, 0.15, 0.15, 0.1, 0.1, 0.07, 0.05, 0.03])
            
            for step_idx in range(n_steps):
                event_name = funnel_steps[step_idx]
                event_type_id = event_master[event_name]
                
                wait = np.random.randint(60, 600) if event_name in ['page_view', 'click_product'] else np.random.randint(10, 60)
                u_time += timedelta(seconds=wait)
                
                # REGLAS DE NEGOCIO ESTRICTAS
                is_viewing = step_idx >= 2 
                is_transac = step_idx >= 5 
                # Blindaje: Si el ID es 9, ES una compra
                current_payment = session_payment if event_type_id == 9 else None
                
                all_data.append({
                    'row_id': row_id,
                    'event_type_id': event_type_id,
                    'user_id': u_id,
                    'session_id': u_session_id,
                    'event_name': event_name,
                    'event_timestamp': u_time,
                    'time_zone': u_tz,
                    'country': u_country,
                    'device': u_device,
                    'traffic_source': u_source,
                    'category': u_category if is_viewing else None,
                    'sku': u_sku if is_viewing else None,
                    'cart_id': u_cart_id if is_transac else None,
                    'payment_method': current_payment, # GARANTIZADO
                    'units': 1 if is_transac else 0,
                    'amount': u_price if is_transac else 0.0
                })
                row_id += 1
                if len(all_data) >= target_rows: break
            if len(all_data) >= target_rows: break
        if len(all_data) >= target_rows: break

    df = pd.DataFrame(all_data)
    return df.sample(frac=1).reset_index(drop=True)

df_final = generate_pro_session_dataset(1550127)
df_final.to_csv('data_clase_sql_final.csv', index=False)

🚀 Generando versión final con Payment Method garantizado en Purchase...


In [141]:
#df_final['payment_method'].value_counts()
#df_final['event_name'].value_counts()
#df_final['category'].value_counts()
#df_final['country'].value_counts()
df_final.head(20)


,row_id,event_type_id,user_id,session_id,event_name,event_timestamp,time_zone,country,device,traffic_source,category,sku,cart_id,payment_method,units,amount
0,345452,5,11036270,SESS-11036270-3,click_product,2026-01-19 15:12:47,UTC-3,Chile,Tablet,Google,Clothing,SKU-1113,None,None,0,0.00
1,617613,1,11064791,SESS-11064791-1,home_page,2026-01-14 23:00:20,UTC+1,España,Tablet,Direct,None,None,None,None,0,0.00
2,740512,4,11077777,SESS-11077777-1,scroll,2026-01-28 03:08:45,UTC-3,Argentina,Tablet,Google,Electronics,SKU-4650,None,None,0,0.00
3,81342,4,11008553,SESS-11008553-2,scroll,2026-01-20 10:10:00,UTC-3,Argentina,Desktop,Email,Electronics,SKU-1960,None,None,0,0.00
4,559780,1,11058733,SESS-11058733-1,home_page,2026-01-02 21:00:27,UTC-6,México,Mobile,Facebook,None,None,None,None,0,0.00
5,802389,1,11084279,SESS-11084279-1,home_page,2026-01-26 16:00:13,UTC-3,Argentina,Mobile,Facebook,None,None,None,None,0,0.00
6,592285,7,11062168,SESS-11062168-1,begin_checkout,2026-01-27 18:15:56,UTC+1,España,Tablet,Direct,Beauty,SKU-4330,CART-827888,None,1,364.63
7,1508068,4,11158071,SESS-11158071-1,scroll,2026-01-19 03:11:58,UTC+1,España,Mobile,Direct,Clothing,SKU-3472,None,None,0,0.00
8,1312976,1,11137674,SESS-11137674-2,home_page,2026-01-17 08:00:39,UTC-3,Chile,Tablet,Email,None,None,None,None,0,0.00
9,862761,4,11090628,SESS-11090628-3,scroll,2026-01-17 19:12:04,UTC-6,México,Desktop,Direct,Clothing,SKU-1766,None,None,0,0.00


In [142]:
df_final[df_final['user_id'] == 11071245].sort_values('event_timestamp', ascending=True)

,row_id,event_type_id,user_id,session_id,event_name,event_timestamp,time_zone,country,device,traffic_source,category,sku,cart_id,payment_method,units,amount
1042953,678673,1,11071245,SESS-11071245-1,home_page,2026-01-02 16:00:50,UTC+1,España,Mobile,Facebook,None,None,None,None,0,0.0
241773,678674,2,11071245,SESS-11071245-1,log_in,2026-01-02 16:01:21,UTC+1,España,Mobile,Facebook,None,None,None,None,0,0.0
236621,678675,3,11071245,SESS-11071245-1,page_view,2026-01-02 16:06:42,UTC+1,España,Mobile,Facebook,Clothing,SKU-4133,None,None,0,0.0
820932,678676,1,11071245,SESS-11071245-2,home_page,2026-01-16 06:00:36,UTC+1,España,Mobile,Facebook,None,None,None,None,0,0.0


In [ ]:
df_final[df_final['user_id'] == 11155007].sort_values('event_timestamp', ascending=True)

,row_id,event_type_id,user_id,session_id,event_name,event_timestamp,time_zone,country,device,traffic_source,category,sku,cart_id,payment_method,units,amount
22515,1477146,1,11155007,SESS-11155007-4,home_page,2026-01-05 19:00:33,UTC-3,Chile,Tablet,Direct,None,None,None,None,0,0.00
929431,1477147,2,11155007,SESS-11155007-4,log_in,2026-01-05 19:01:15,UTC-3,Chile,Tablet,Direct,None,None,None,None,0,0.00
858058,1477148,3,11155007,SESS-11155007-4,page_view,2026-01-05 19:08:57,UTC-3,Chile,Tablet,Direct,Home,SKU-2173,None,None,0,0.00
250849,1477149,4,11155007,SESS-11155007-4,scroll,2026-01-05 19:09:33,UTC-3,Chile,Tablet,Direct,Home,SKU-2173,None,None,0,0.00
32857,1477150,5,11155007,SESS-11155007-4,click_product,2026-01-05 19:11:56,UTC-3,Chile,Tablet,Direct,Home,SKU-2173,None,None,0,0.00
1352761,1477151,6,11155007,SESS-11155007-4,add_to_cart,2026-01-05 19:12:15,UTC-3,Chile,Tablet,Direct,Home,SKU-2173,CART-393871,None,1,212.38
370181,1477127,1,11155007,SESS-11155007-1,home_page,2026-01-14 10:00:30,UTC-3,Chile,Tablet,Direct,None,None,None,None,0,0.00
398512,1477128,2,11155007,SESS-11155007-1,log_in,2026-01-14 10:00:45,UTC-3,Chile,Tablet,Direct,None,None,None,None,0,0.00
234314,1477129,3,11155007,SESS-11155007-1,page_view,2026-01-14 10:07:22,UTC-3,Chile,Tablet,Direct,Electronics,SKU-3268,None,None,0,0.00
265518,1477130,4,11155007,SESS-11155007-1,scroll,2026-01-14 10:07:59,UTC-3,Chile,Tablet,Direct,Electronics,SKU-3268,None,None,0,0.00


In [ ]:
import pandas as pd
import sqlite3
import time
from tqdm.notebook import tqdm
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML, clear_output

# 1. Configuración de enlaces (URL LFS Directa)
datasets = {
    'ecommerce_events': "https://media.githubusercontent.com/media/hector1994/e_commerce_funnel_data_generator/refs/heads/master/data_clase_sql_final.csv"
}

# 2. Conexión a la base de datos en memoria
# Aumentamos el timeout para procesar el gran volumen de datos
connector = sqlite3.connect(':memory:', check_same_thread=False)

# 3. Proceso de carga con Barra de Progreso
print("⬇️🗂️ Descargando y sincronizando Dataset Masivo (1.5M registros).....🔄⚙️")
summary_data = []

for name, url in tqdm(datasets.items(), desc="Cargando Tablas"):
    # Descarga de datos
    df = pd.read_csv(url)

    # --- CAMBIO IMPORTANTE: DEFINICIÓN DE FECHAS ---
    # Convertimos la columna de timestamp a formato datetime real
    if 'event_timestamp' in df.columns:
        df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])

    # Conversión a SQL (con chunksize para optimizar memoria)
    df.to_sql(name, connector, index=False, if_exists='replace', chunksize=10000)

    # Guardar info para el resumen
    summary_data.append({
        "Table Name": f"{name}",
        "Rows": f"{len(df):,}",
        "Columns": len(df.columns)
    })

# 4. Definición de la "Palabra mágica" %%sql
@register_cell_magic
def sql(line, cell):
    try:
        # Ejecución de la consulta
        resultado = pd.read_sql(cell, connector)
        clear_output(wait=True)
        display(HTML("<b style='color: #4CAF50;'>✅ Query completed successfully:</b>"))
        return display(resultado)
    except Exception as e:
        clear_output(wait=True)
        display(HTML(f"<b style='color: #F44336;'>❌ Query execution failed:</b><br><code style='color: grey;'>{str(e)}</code>"))

# 5. Interfaz final para el alumno
clear_output()
display(HTML("<h2 style='color: #8e62f3'>✨ Database initialized - E-commerce Funnel Data ✨</h2>"))
display(HTML("<p>The 1.5M rows dataset has been loaded. You can now use <b>ecommerce_events</b> in your queries:</p>"))

# Mostramos el resumen
display(pd.DataFrame(summary_data))
print("\n ✨🚀 SYSTEM READY! START ANALYZING THE FUNNEL! 🚀✨")

,Table Name,Rows,Columns
0,ecommerce_events,"1,550,127",15



 ✨🚀 SYSTEM READY! START ANALYZING THE FUNNEL! 🚀✨


In [ ]:
%%sql
SELECT *
FROM ecommerce_events 
ORDER BY event_timestamp DESC, user_id DESC;


,row_id,event_type_id,user_id,session_id,event_name,event_timestamp,time_zone,country,device,traffic_source,category,sku,cart_id,units,amount
0,1468153,9,11153839,SESS-11153839-4,purchase,2026-01-31 23:22:17,UTC-3,Argentina,Desktop,Google,Clothing,SKU-1098,CART-632246,1,142.42
1,1468152,8,11153839,SESS-11153839-4,end_checkout,2026-01-31 23:21:45,UTC-3,Argentina,Desktop,Google,Clothing,SKU-1098,CART-632246,1,142.42
2,1468151,7,11153839,SESS-11153839-4,begin_checkout,2026-01-31 23:21:09,UTC-3,Argentina,Desktop,Google,Clothing,SKU-1098,CART-632246,1,142.42
3,676891,9,11070884,SESS-11070884-3,purchase,2026-01-31 23:20:36,UTC-6,México,Tablet,Facebook,Home,SKU-2230,CART-289370,1,221.06
4,1210007,7,11126719,SESS-11126719-3,begin_checkout,2026-01-31 23:20:30,UTC+1,España,Mobile,Email,Home,SKU-3689,CART-828870,1,173.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550122,693878,1,11072700,SESS-11072700-3,home_page,2026-01-01 00:00:10,UTC-3,Chile,Mobile,Email,None,None,None,0,0.00
1550123,618230,1,11064717,SESS-11064717-1,home_page,2026-01-01 00:00:10,UTC-3,Argentina,Tablet,Google,None,None,None,0,0.00
1550124,579174,1,11060596,SESS-11060596-1,home_page,2026-01-01 00:00:10,UTC-3,Chile,Desktop,Facebook,None,None,None,0,0.00
1550125,520383,1,11054486,SESS-11054486-1,home_page,2026-01-01 00:00:10,UTC-6,México,Mobile,Google,None,None,None,0,0.00


In [156]:
import plotly.express as px

# 1. Datos agrupados
funnel_order = ['home_page', 'log_in', 'page_view', 'scroll', 'click_product', 
                'add_to_cart', 'begin_checkout', 'end_checkout', 'purchase']

funnel_data = df_final.groupby('event_name')['user_id'].nunique().reset_index()
funnel_data['event_name'] = pd.Categorical(funnel_data['event_name'], categories=funnel_order, ordered=True)
funnel_data = funnel_data.sort_values('event_name')

# 2. Creación con paleta Aesthetic
fig = px.funnel(
    funnel_data, 
    x='user_id', 
    y='event_name',
    color='event_name',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# 3. Ajustes de Estilo y Eliminación de Negativos
fig.update_traces(
    textposition='inside',
    textinfo='value+percent initial',
    marker_line_width=1,
    marker_line_color='white',
    opacity=0.9
)

fig.update_layout(
    title={
        'text': "<b>ESTRUCTURA DE CONVERSIÓN (%)</b><br><span style='font-size:25px; color:#95a5a6;'>Volumen de Usuarios Únicos por Etapa</span>",
        'y': 0.95, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'
    },
    paper_bgcolor='#FFFFFF', 
    plot_bgcolor='#FFFFFF',
    font=dict(family="Arial, sans-serif", size=20, color="#515A5A"),
    showlegend=False,
    width=1000,
    height=700,
    margin=dict(l=200, r=100, t=120, b=80)
)

# --- SOLUCIÓN AL EJE X ---
fig.update_xaxes(
    showgrid=True, 
    gridcolor='#F0F0F0', 
    showline=True,
    linecolor='#DCDCDC',
    # Quitamos los números (ticklabels) para evitar los negativos
    showticklabels=False, 
    # Mantenemos el título para dar contexto
    title="Distribución de Volumen Total",
    title_font=dict(size=11, color='#BDC3C7'),
    zeroline=False # Quitamos la línea del cero central para máxima limpieza
)

fig.update_yaxes(
    showline=True,
    linecolor='#BDC3C7',
    linewidth=1,
    title=None
)

fig.show()